In [94]:
import compute_convergence
import data
import glob
import seaborn as sns
import numpy as np
import pandas as pd

sns.set_theme()

# Data for experiments

In [95]:
biased = data.load_data("../output/biased/synthetic/s12/")
hybrid = data.load_data("../output/hybrid/synthetic/s12/")
#biased = data.postprocess(biased)

In [96]:
biased[biased.method != "EMCC"]

,index,treewdith,bias_endo_treewdith,method,endo_treewdith,bias_treewdith,model_path,ps1,ll_max,ace_l,...,llk_101,llk_102,sparents,file,network_type,mc,n,mid,d,inf
0,0,3.0,NaN,exact_precise,3.0,NaN,/mnt/beegfs/rcabanas/dev/credici/papers/journa...,1.0,-1981.064670,NaN,...,NaN,NaN,NaN,../output/biased/synthetic/s12/random_mc2_n5_m...,random,2,5,3,1000,wtrue
11,0,3.0,NaN,exact_precise,3.0,NaN,/mnt/beegfs/rcabanas/dev/credici/papers/journa...,1.0,-4813.201345,NaN,...,NaN,NaN,NaN,../output/biased/synthetic/s12/random_mc2_n6_m...,random,2,6,3,1000,wtrue
12,1,3.0,NaN,exact_data_based,3.0,NaN,/mnt/beegfs/rcabanas/dev/credici/papers/journa...,1.0,-4813.201345,NaN,...,NaN,NaN,NaN,../output/biased/synthetic/s12/random_mc2_n6_m...,random,2,6,3,1000,wtrue
23,0,3.0,NaN,exact_precise,3.0,NaN,/mnt/beegfs/rcabanas/dev/credici/papers/journa...,1.0,-2323.286204,NaN,...,NaN,NaN,NaN,../output/biased/synthetic/s12/random_mc2_n7_m...,random,2,7,3,1000,wtrue
24,1,3.0,NaN,exact_data_based,3.0,NaN,/mnt/beegfs/rcabanas/dev/credici/papers/journa...,1.0,-2323.286204,NaN,...,NaN,NaN,NaN,../output/biased/synthetic/s12/random_mc2_n7_m...,random,2,7,3,1000,wtrue
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2424,1,2.0,NaN,exact_data_based,2.0,NaN,/mnt/beegfs/rcabanas/dev/credici/papers/journa...,1.0,-1648.601552,NaN,...,NaN,NaN,NaN,../output/biased/synthetic/s12/random_mc2_n5_m...,random,2,5,3,1000,wtrue
2435,0,3.0,NaN,exact_precise,3.0,NaN,/mnt/beegfs/rcabanas/dev/credici/papers/journa...,1.0,-1937.342221,NaN,...,NaN,NaN,NaN,../output/biased/synthetic/s12/random_mc2_n5_m...,random,2,5,3,1000,wtrue
2446,0,2.0,NaN,exact_precise,2.0,NaN,/mnt/beegfs/rcabanas/dev/credici/papers/journa...,1.0,-1234.055317,NaN,...,NaN,NaN,NaN,../output/biased/synthetic/s12/random_mc2_n5_m...,random,2,5,3,1000,wtrue
2447,1,2.0,NaN,exact_data_based,2.0,NaN,/mnt/beegfs/rcabanas/dev/credici/papers/journa...,1.0,-1234.055317,NaN,...,NaN,NaN,NaN,../output/biased/synthetic/s12/random_mc2_n5_m...,random,2,5,3,1000,wtrue


In [97]:
print(biased.shape, hybrid.shape)

(2469, 924) (364, 625)


In [98]:
biased.groupby("method").agg("index").count()

method
EMCC                2150
exact_data_based     104
exact_precise        215
Name: index, dtype: int64

In [99]:
a = np.array(biased.columns)
a.sort()


In [100]:
biased[(biased.ps1 == 1)&(biased.method == 'EMCC')].file

1       ../output/biased/synthetic/s12/random_mc2_n5_m...
10      ../output/biased/synthetic/s12/random_mc2_n5_m...
13      ../output/biased/synthetic/s12/random_mc2_n6_m...
22      ../output/biased/synthetic/s12/random_mc2_n6_m...
25      ../output/biased/synthetic/s12/random_mc2_n7_m...
                              ...                        
2445    ../output/biased/synthetic/s12/random_mc2_n5_m...
2448    ../output/biased/synthetic/s12/random_mc2_n5_m...
2457    ../output/biased/synthetic/s12/random_mc2_n5_m...
2459    ../output/biased/synthetic/s12/random_mc2_n6_m...
2468    ../output/biased/synthetic/s12/random_mc2_n6_m...
Name: file, Length: 436, dtype: object

Filter cases where ratio > 0.95 and save to pickle

In [102]:
dta = biased.copy()
print(dta.shape)
dta['ll'] = dta.apply(lambda row: row[[f"llk_{i}" for i in range(300)]].values, axis=1)
dta["pns"]= dta.apply(lambda row: row[[f"pns_{i}" for i in range(300)]].values, axis=1)

dta = dta[np.abs(dta.pns_l - dta.pns_u) > 0.02]
outdata_biased=dta[['pns', 'll', 'll_max', "pns_l", "pns_u", 'file', 'selector', "ps1", "method"]]

(2469, 924)


In [103]:
outdata_biased.to_json("pns_12_biased.json")

In [93]:
a = np.array(hybrid.columns)
a.sort()

In [104]:
dta = hybrid.copy()
print(dta.shape)
#dta['ll'] = dta.apply(lambda row: row[[f"llk_{i}" for i in range(300)]].values, axis=1)
dta["pns"]= dta.apply(lambda row: row[[f"pns_{i}" for i in range(300)]].values, axis=1)
dta['pns_l'] = dta.pns.apply(min)
dta['pns_u'] = dta.pns.apply(max)
dta = dta[np.abs(dta.pns_l - dta.pns_u) > 0.02]
outdata_hybrid=dta[['pns', "pns_l", "pns_u", 'file', "model_type",]]
hybrid.model_type.unique()

(364, 625)


array(['Dobs+Dxb', 'Dobs+Dx+Dxb ', 'Dx+Dxb', 'Dxb'], dtype=object)

In [105]:
outdata_hybrid.to_json("pns_12_hybrid.json")

In [106]:
import convergence